<img style="float: right" src="images/surfsara.png">
<br/>
<hr style="clear: both" />

# Spark Structured API: DataFrames and SQL
In the previous notebook you have seen distributed processing using RDDs is done. In this notebook we will look at Spark's Structured API. We will see how you can use DataFrames and SQL to do common data processing operations. By the end you should have a feeling for the strengths and weaknesses of these different approaches.

**Please complete the assignments in this notebook, and download your notebook: 'File' -> 'Download' -> 'Notebook (.ipynb)'. Send this notebook to [`helpdesk@surfsara.nl`](mailto:helpdesk@surfsara.nl), with your name. Please mention 'UvA HPC course' in your email subject.**

The first difference is our Spark _entrypoint_. For RDDs this was the 'SparkContext' (usually named `sc`). For DataFrames we will use a 'SparkSession', which is more powerful and easier to use. By convention we name our SparkSession `spark`, and we create it as follows:

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

We can use a SparkSession to create DataFrames (as we will soon see) and these can be converted to RDDs. However if we directly want to create RDDs we have to do this via SparkContext. A SparkContext is contained in SparkSession, and can be used as follows:

In [2]:
sc = spark.sparkContext
rdd = sc.parallelize(['a', 'b', 'c'])
rdd.collect()

['a', 'b', 'c']

## DataFrames from Python collections

Just like we have seen with `sc.parallelize` for RDDs, we can create a DataFrame from an existing Python collection. In addition to the collection itself we will also describe (part of) the structure of the data by naming the columns. Additionally, we could  specify the data types of the columns, but in this case we can let Spark infer this automatically.

First, a list of tuples in Python is created, called `phone_stock`. Next, we create a list called `columns` that contain the name of all columns of the DataFrame. Then we use these two lists as input for [`createDataFrame`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SparkSession.createDataFrame). The result is the DataFrame `phone_df`. Next we print the type of both `phone_stock` and `phone_df`.

In [3]:
phone_stock = [
    ('iPhone 6', 'Apple', 6, 549.00),
    ('iPhone 6s', 'Apple', 5, 585.00),
    ('iPhone 7', 'Apple', 11, 739.00),
    ('Pixel', 'Google', 8, 859.00),
    ('Pixel XL', 'Google', 2, 959.00),
    ('Galaxy S7', 'Samsung', 10, 539.00),
    ('Galaxy S6', 'Samsung', 5, 414.00),
    ('Galaxy A5', 'Samsung', 7, 297.00),
    ('Galaxy Note 7', 'Samsung', 0, 841.00)
]

columns = ['model', 'brand', 'stock', 'unit_price']

phone_df = spark.createDataFrame(phone_stock, columns)

print('the type of phoneStock: ' + str(type(phone_stock)))
print('the type of phone_df: ' + str(type(phone_df)))

the type of phoneStock: <class 'list'>
the type of phone_df: <class 'pyspark.sql.dataframe.DataFrame'>


In order to see a few rows of a DataFrame use [`show()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.show). By default it shows 20 rows, but you can give the desired number of rows that you want to see as an argument.

In [5]:
phone_df.show()

+-------------+-------+-----+----------+
|        model|  brand|stock|unit_price|
+-------------+-------+-----+----------+
|     iPhone 6|  Apple|    6|     549.0|
|    iPhone 6s|  Apple|    5|     585.0|
|     iPhone 7|  Apple|   11|     739.0|
|        Pixel| Google|    8|     859.0|
|     Pixel XL| Google|    2|     959.0|
|    Galaxy S7|Samsung|   10|     539.0|
|    Galaxy S6|Samsung|    5|     414.0|
|    Galaxy A5|Samsung|    7|     297.0|
|Galaxy Note 7|Samsung|    0|     841.0|
+-------------+-------+-----+----------+



Like RDDs we have a [`collect()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.collect) action that returns all data from a DataFrame to the driver. Notice that we get `Row` objects that contain column name and value pairs. Remember that the result of a `collect()` is a Python data structure (a list of `Row` objects).

In [6]:
all_phones = phone_df.collect()
all_phones

[Row(model='iPhone 6', brand='Apple', stock=6, unit_price=549.0),
 Row(model='iPhone 6s', brand='Apple', stock=5, unit_price=585.0),
 Row(model='iPhone 7', brand='Apple', stock=11, unit_price=739.0),
 Row(model='Pixel', brand='Google', stock=8, unit_price=859.0),
 Row(model='Pixel XL', brand='Google', stock=2, unit_price=959.0),
 Row(model='Galaxy S7', brand='Samsung', stock=10, unit_price=539.0),
 Row(model='Galaxy S6', brand='Samsung', stock=5, unit_price=414.0),
 Row(model='Galaxy A5', brand='Samsung', stock=7, unit_price=297.0),
 Row(model='Galaxy Note 7', brand='Samsung', stock=0, unit_price=841.0)]

Working directly with a list of row objects is cumbersome. To work directly with data on the driver's side, we usually convert the Spark DataFrame to a `pandas` DataFrame. [`pandas`](https://pandas.pydata.org/) is a data processing library that allows us to manipulate tabular table. It is suitable for processing that isn't too intensive and data that isn't too large to fit into local memory (otherwise, why would we want to use Spark?).

Spark DataFrames have a [`toPandas()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.toPandas) action defined on them, that will pull all data to the driver and convert it to a `pandas` DataFrame:

In [7]:
phone_df.toPandas()

,model,brand,stock,unit_price
0,iPhone 6,Apple,6,549.0
1,iPhone 6s,Apple,5,585.0
2,iPhone 7,Apple,11,739.0
3,Pixel,Google,8,859.0
4,Pixel XL,Google,2,959.0
5,Galaxy S7,Samsung,10,539.0
6,Galaxy S6,Samsung,5,414.0
7,Galaxy A5,Samsung,7,297.0
8,Galaxy Note 7,Samsung,0,841.0


There are several ways to look at the structure of a DataFrame: `printSchema`, `schema` and `describe`. [`printSchema`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.printSchema) is especially useful with complicated nested structures, because it provides a human-readable form:

In [8]:
phone_df.printSchema()

root
 |-- model: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- stock: long (nullable = true)
 |-- unit_price: double (nullable = true)



Note that all columns are listed, together with their type and a boolean value that indicates whether the value for that column can be NULL.

Schema's can also be listed programmatically. By calling [`schema`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.schema) we get to see the structure of the DataFrame in Sparks types. It is possible to define a schema in code by making use of these types, although we won't do this here.

In [9]:
phone_df.schema

StructType(List(StructField(model,StringType,true),StructField(brand,StringType,true),StructField(stock,LongType,true),StructField(unit_price,DoubleType,true)))

It is also possible to look more closely on the structure of fields, in which the columns are defined:

In [10]:
phone_df.schema.fields

[StructField(model,StringType,true),
 StructField(brand,StringType,true),
 StructField(stock,LongType,true),
 StructField(unit_price,DoubleType,true)]

[`describe`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe) will compute summary statistics for numeric and string columns:

In [11]:
phone_df.describe().show()

+-------+---------+-------+------------------+------------------+
|summary|    model|  brand|             stock|        unit_price|
+-------+---------+-------+------------------+------------------+
|  count|        9|      9|                 9|                 9|
|   mean|     null|   null|               6.0| 642.4444444444445|
| stddev|     null|   null|3.5355339059327378|220.82295573100586|
|    min|Galaxy A5|  Apple|                 0|             297.0|
|    max| iPhone 7|Samsung|                11|             959.0|
+-------+---------+-------+------------------+------------------+



## Data extraction

Now that we have our data in a DataFrame, we want to use it to manipulate the data. Let's start by selecting subsets of the data: specific columns and/or rows.

### Selecting columns

Often we are not interested in all the columns of our data. DataFrames make it very easy to select only a subset by using the [`select`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe) method. Realise that we are not modifying the original DataFrame, but creating a new one.

In [12]:
# Select only the model column
model_df = phone_df.select("model")
model_df.show()

+-------------+
|        model|
+-------------+
|     iPhone 6|
|    iPhone 6s|
|     iPhone 7|
|        Pixel|
|     Pixel XL|
|    Galaxy S7|
|    Galaxy S6|
|    Galaxy A5|
|Galaxy Note 7|
+-------------+



We can also rename a column by using [`expr`](https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#pyspark.sql.functions.expr).

In [13]:
from pyspark.sql.functions import expr
mymodel_df = phone_df.select("brand", expr("model as mymodel"))
mymodel_df.show()

+-------+-------------+
|  brand|      mymodel|
+-------+-------------+
|  Apple|     iPhone 6|
|  Apple|    iPhone 6s|
|  Apple|     iPhone 7|
| Google|        Pixel|
| Google|     Pixel XL|
|Samsung|    Galaxy S7|
|Samsung|    Galaxy S6|
|Samsung|    Galaxy A5|
|Samsung|Galaxy Note 7|
+-------+-------------+



In [14]:
# Select both the brand and model columns
bm_df = phone_df.select('brand', 'model')
bm_df.show()

+-------+-------------+
|  brand|        model|
+-------+-------------+
|  Apple|     iPhone 6|
|  Apple|    iPhone 6s|
|  Apple|     iPhone 7|
| Google|        Pixel|
| Google|     Pixel XL|
|Samsung|    Galaxy S7|
|Samsung|    Galaxy S6|
|Samsung|    Galaxy A5|
|Samsung|Galaxy Note 7|
+-------+-------------+



## Assignment 1
Select the `model` and `stock` columns from `phone_df`:

In [15]:
# TODO: Replace <FILL IN> with appropriate code
# Select the model and stock columns
ms_df = phone_df.select('model','stock')
ms_df.show()

+-------------+-----+
|        model|stock|
+-------------+-----+
|     iPhone 6|    6|
|    iPhone 6s|    5|
|     iPhone 7|   11|
|        Pixel|    8|
|     Pixel XL|    2|
|    Galaxy S7|   10|
|    Galaxy S6|    5|
|    Galaxy A5|    7|
|Galaxy Note 7|    0|
+-------------+-----+



### Filtering rows

We can filter specific rows by using the DataFrame [`filter`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.filter) method. Please note that the [`where`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where) method is an alias for `filter`. The column specifications are the same as with the select method:

In [ ]:
# Select rows with phones from Google
google_df = phone_df.filter(phone_df['brand'] == 'Google')

google_df.show()

## Assignment 2
Select the rows with `unit_price` less than 550.00

In [16]:
# TODO: Replace <FILL IN> with appropriate code

cheap_df = phone_df.filter(phone_df['unit_price'] < 550.00)
cheap_df.show()

+---------+-------+-----+----------+
|    model|  brand|stock|unit_price|
+---------+-------+-----+----------+
| iPhone 6|  Apple|    6|     549.0|
|Galaxy S7|Samsung|   10|     539.0|
|Galaxy S6|Samsung|    5|     414.0|
|Galaxy A5|Samsung|    7|     297.0|
+---------+-------+-----+----------+



Multiple filter conditions can be specified using Python's [boolean operations](https://docs.python.org/3/library/stdtypes.html#boolean-operations-and-or-not):

In [17]:
phone_df.filter((phone_df.brand == 'Apple') | (phone_df.brand == 'Google')).show()

+---------+------+-----+----------+
|    model| brand|stock|unit_price|
+---------+------+-----+----------+
| iPhone 6| Apple|    6|     549.0|
|iPhone 6s| Apple|    5|     585.0|
| iPhone 7| Apple|   11|     739.0|
|    Pixel|Google|    8|     859.0|
| Pixel XL|Google|    2|     959.0|
+---------+------+-----+----------+



### Ordering rows

We can use the [`orderBy`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy) method to sort data:

In [18]:
phone_df.orderBy('unit_price').show()

+-------------+-------+-----+----------+
|        model|  brand|stock|unit_price|
+-------------+-------+-----+----------+
|    Galaxy A5|Samsung|    7|     297.0|
|    Galaxy S6|Samsung|    5|     414.0|
|    Galaxy S7|Samsung|   10|     539.0|
|     iPhone 6|  Apple|    6|     549.0|
|    iPhone 6s|  Apple|    5|     585.0|
|     iPhone 7|  Apple|   11|     739.0|
|Galaxy Note 7|Samsung|    0|     841.0|
|        Pixel| Google|    8|     859.0|
|     Pixel XL| Google|    2|     959.0|
+-------------+-------+-----+----------+



#### Note: Columns specifications

In the previous examples we have used various _column specifications_ for selecting and filtering data. Sometimes the more complicated ones are required because the shorter versions are ambiguous for Spark's parser. For example, all these are equivalent:

```
bm_df = phone_df.select("brand", "model")
bm_df = phone_df.select(["brand", "model"])
bm_df = phone_df.select(phone_df["brand"], phone_df["model"])
```

In the next cell we use a chain of DataFrame methods that are very similar to the SQL query language used for certain databases.
    Notice that we use only the names of columns. Note, the use of double and single quotes in the [`where`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where) method.

In [19]:
phone_df.select("model", "unit_price").where("brand='Apple'").orderBy('stock', ascending=False).show()

+---------+----------+
|    model|unit_price|
+---------+----------+
| iPhone 7|     739.0|
| iPhone 6|     549.0|
|iPhone 6s|     585.0|
+---------+----------+



An alternative way of doing the same as the cell above is using `phone_df["brand"]` in the where clause. This is longer to type but intuitively more clear and easier to read. There is no ambiguity for the Spark parser with this notation.

In [20]:
phone_df.select("model", "unit_price").where(phone_df["brand"]=="Apple").orderBy('stock', ascending=False).show()

+---------+----------+
|    model|unit_price|
+---------+----------+
| iPhone 7|     739.0|
| iPhone 6|     549.0|
|iPhone 6s|     585.0|
+---------+----------+



## Assignment 3
Select all phones with a unit price larger than 300 and of which there are more than two in stock. Display the remaining phones, ordered by brand, followed by stock. Use whatever column specification syntax you prefer.

In [32]:
phone_df.filter((phone_df["unit_price"] > 300) & (phone_df["stock"]>2)).orderBy("brand","stock").show()

+---------+-------+-----+----------+
|    model|  brand|stock|unit_price|
+---------+-------+-----+----------+
|iPhone 6s|  Apple|    5|     585.0|
| iPhone 6|  Apple|    6|     549.0|
| iPhone 7|  Apple|   11|     739.0|
|    Pixel| Google|    8|     859.0|
|Galaxy S6|Samsung|    5|     414.0|
|Galaxy S7|Samsung|   10|     539.0|
+---------+-------+-----+----------+



## Aggregating data
An important part of data processing is the ability to combine multiple records, like we did with `reduceByKey`. In the DataFrame API this is a two-step process:

First you group the data using the `groupBy` method. `groupBy` can operate on one or multiple columns. It will not actually perform the grouping but create a reference to a `GroupedData` object:

In [33]:
grouped_df = phone_df.groupBy('brand')
print(type(grouped_df))

<class 'pyspark.sql.group.GroupedData'>


After the data is grouped we can apply one of the standard aggregation functions on it. They are listed at the [GroupedData](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) API documentation. These are: `min`, `max`, `mean`, `sum` and `count`. We can apply an aggregation to all columns or to a subset of the columns.

In [34]:
# Minimum for all columns
min_df = grouped_df.min('unit_price')

min_df.toPandas()

,brand,min(unit_price)
0,Samsung,297.0
1,Google,859.0
2,Apple,549.0


Notice that the `min(unit_price)` is the name of the new column. If you want to rename a column use [`withColumnRenamed`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.withColumnRenamed). As arguments this method takes the old name and new name of the column.

## Assignment 4

Compute the maximum  of the unit_price per brand and rename the resulting column to `max`.
(We assume you can do this in one line. Feel free to adapt the cell and use more lines if you want.)

In [35]:
# TODO: Replace <FILL IN> with appropriate code
max_df = grouped_df.max('unit_price').withColumnRenamed('max(unit_price)','max')
max_df.toPandas()

,brand,max
0,Samsung,841.0
1,Google,959.0
2,Apple,739.0


Finally, we can combine different aggregations per column using the [`agg`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.agg) method on a GroupedData instance:

In [36]:
# Take the sum of the stock column, and calculate the mean of the unit_price column, in one go
sum_df = grouped_df.agg({'stock': 'sum', 'unit_price': 'mean'})

sum_df.show()

+-------+-----------------+----------+
|  brand|  avg(unit_price)|sum(stock)|
+-------+-----------------+----------+
|Samsung|           522.75|        22|
| Google|            909.0|        10|
|  Apple|624.3333333333334|        22|
+-------+-----------------+----------+



## SQL
The SQL API aims to be ANSI-SQL SQL2003 and Hive-SQL compatible. The expressiveness is very similar to the DataFrame API. You can access the SQL API from the SparkSession by using `spark.sql`. Below is a query performed using Spark's DataFrame API:

In [37]:
# DataFrame version
res_df = phone_df.filter(phone_df['stock'] > 7).select('model')
res_df.show()

+---------+
|    model|
+---------+
| iPhone 7|
|    Pixel|
|Galaxy S7|
+---------+



The SQL version of the query requires us to 'register' the DataFrame as an SQL table: 

In [38]:
# SQL version

# Register the phone_df DataFrame within SQL as a table with name 'phones'
phone_df.createOrReplaceTempView('phones')

# Perform the SQL query on the 'phones' table
res_df = spark.sql('SELECT model FROM phones WHERE stock > 7')
res_df.show()

+---------+
|    model|
+---------+
| iPhone 7|
|    Pixel|
|Galaxy S7|
+---------+



## Joining with other data sets
Often you want to combine multiple datasets on a shared column. In this example we create an extra table with information about the phone manufacturer:

In [39]:
companies = [
    ('Google', 'USA', 1998, 'Sundar Pichai'),
    ('Samsung', 'South Korea', 1938 ,'Oh-Hyun Kwon' ),
    ('Apple', 'USA', 1976 ,'Tim Cook')
]

columns = ['company_name', 'hq_country', 'founding_year', 'ceo']

company_df = spark.createDataFrame(companies, columns)
company_df.show()

+------------+-----------+-------------+-------------+
|company_name| hq_country|founding_year|          ceo|
+------------+-----------+-------------+-------------+
|      Google|        USA|         1998|Sundar Pichai|
|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|       Apple|        USA|         1976|     Tim Cook|
+------------+-----------+-------------+-------------+



To join two DataFrames, we use the `join` method on one of the DataFrames. This method takes two arguments: (1) the other DataFrame, and (2) a join relation. Here we join the two DataFrames on the brand/company_name columns:

In [40]:
joined_df = phone_df.join(company_df, phone_df['brand'] == company_df['company_name'])
joined_df.show()

+-------------+-------+-----+----------+------------+-----------+-------------+-------------+
|        model|  brand|stock|unit_price|company_name| hq_country|founding_year|          ceo|
+-------------+-------+-----+----------+------------+-----------+-------------+-------------+
|    Galaxy S7|Samsung|   10|     539.0|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|    Galaxy S6|Samsung|    5|     414.0|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|    Galaxy A5|Samsung|    7|     297.0|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|Galaxy Note 7|Samsung|    0|     841.0|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|        Pixel| Google|    8|     859.0|      Google|        USA|         1998|Sundar Pichai|
|     Pixel XL| Google|    2|     959.0|      Google|        USA|         1998|Sundar Pichai|
|     iPhone 6|  Apple|    6|     549.0|       Apple|        USA|         1976|     Tim Cook|
|    iPhone 6s|  Apple|    5|     585.0|       Apple|       

Here is an example of a more complicated query that combines multiple steps:

In [41]:
# All the models from USA companies with more than 7 items in stock
result = phone_df \
    .join(company_df, phone_df['brand'] == company_df['company_name']) \
    .filter(company_df['hq_country'] == 'USA') \
    .filter(phone_df['stock'] > 7) \
    .select('model')

result.show()

+--------+
|   model|
+--------+
|   Pixel|
|iPhone 7|
+--------+



## Assignment 5

The problem below was taken from Coursera's MOOC [Big Data Analysis with Scala and Spark](https://www.coursera.org/learn/scala-spark-big-data) by the École Polytechnique Fédérale de Lausanne. We adapted the problem for PySpark.

Let's assume we have a dataset with posts from a discussion forum. The entries of the dataset consist of an authorID, the name of a subforum, the number of likes and a date. The data frame is constructed in the following cell.

**We would like to know how many likes each author posted on each subforum. The table should show per subforum how many likes each author has, the highest number of likes first.**

In [42]:
from  pyspark.sql import Row
from pyspark.sql.functions import count


posts = [{'authorID' : 4, 'subforum': 'java', 'likes': 5, 'date' : 'sept 5'},
         {'authorID' : 1, 'subforum': 'python', 'likes': 3, 'date' : 'sept 4'},
        {'authorID' : 2, 'subforum': 'python', 'likes': 35, 'date' : 'sept 3'},
        {'authorID' : 3, 'subforum': 'java', 'likes': 1, 'date' : 'sept 5'},
        {'authorID' : 4, 'subforum': 'java', 'likes': 14, 'date' : 'sept 5'},
        {'authorID' : 3, 'subforum': 'python', 'likes': 12, 'date' : 'sept 3'},
        {'authorID' : 3, 'subforum': 'java', 'likes': 14, 'date' : 'sept 5'},
        {'authorID' : 3, 'subforum': 'java', 'likes': 10, 'date' : 'sept 5'},
        {'authorID' : 2, 'subforum': 'python', 'likes': 21, 'date' : 'sept 5'}]

rdd = spark.sparkContext.parallelize(posts)
df_posts = spark.createDataFrame(rdd.map(lambda x : Row(**x)))

Please use a [groupBy](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy), the [sum aggregation function](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.sum) and an [orderBy](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy) to come up with the desired dataFrame. Note that you want to order in descending order.
Also note, that you can use [`groupBy`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) and `orderBy` on more than one column.

If you get confused, break the problem into steps.

In [50]:
df_posts.groupBy("authorID","subforum").sum("likes").withColumnRenamed('sum(likes)','# of likes').orderBy('# of likes', ascending=False).show()

+--------+--------+----------+
|authorID|subforum|# of likes|
+--------+--------+----------+
|       2|  python|        56|
|       3|    java|        25|
|       4|    java|        19|
|       3|  python|        12|
|       1|  python|         3|
+--------+--------+----------+



## Conversion to/from RDD

Sometimes you want to do data manipulations which would be very easy with RDD operations, but complicated with the DataFrame API. Fortunately you can convert between DataFrames and RDDs of type 'Row'. Going from DataFrame to RDD is quite simple. Going back from RDD to DataFrame is more difficult because you need to re-apply the schema.

In [51]:
phone_rdd = phone_df.rdd
plural_rdd = phone_rdd.map(lambda r: r.brand + 's')
plural_rdd.collect()

['Apples',
 'Apples',
 'Apples',
 'Googles',
 'Googles',
 'Samsungs',
 'Samsungs',
 'Samsungs',
 'Samsungs']

# Reading structured files/sources
One of the advantages of DataFrames is the ability to read already structured data and automatically import the structure in Spark. Spark contains readers for a number of formats such as csv, json, parquet, orc, text and jdbc. There are also third-party readers/connectors for databases such as MongoDB and Cassandra.

Here we read the json-formatted tweets that we also used in the last notebook. As you can see the complicated JSON schema is inferred.

In [52]:
tweet_df = spark.read.format("json").load('../data/tweets.json')
tweet_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- media_url: string (nullable = true)
 |    |    |    |-- media_url_https: string (nullable = true)
 |    |    |    |-- s

This structure is squeezed into a table. When we convert to Pandas we can see what the first tweet looks like in a DataFrame.

In [53]:
tweet_df.toPandas().head(1)

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,None,None,Wed Apr 29 13:26:48 +0000 2015,"([], None, [], [], [], [(48305190, 48305190, [...",None,0,False,low,None,593406077439516672,...,None,False,0,False,None,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@OdekedeJong Omdat ik het zelf ook ervaar en m...,1430314008470,False,"(False, Thu Mar 04 11:16:36 +0000 2010, False,..."


## Assignment 6
Select the name and screen_name of the user, the text field and the lang field.

**Hint**: nested fields can be selected using the dot notation, i.e. `df.select('<parent>.<child>')`.

In [59]:
name_df = tweet_df.select('user.name','user.screen_name','text','lang')
name_df.toPandas().head(15)

,name,screen_name,text,lang
0,Claudia,Claudia_NL,@OdekedeJong Omdat ik het zelf ook ervaar en m...,nl
1,Monique Huijdink,MiesjeB,"RT @RHoogland: The game is on, vrienden. Vrijd...",nl
2,Martine van Dijk,martine_vandijk,@deBeschaving Snap ik! Ben nou eenmaal wat ong...,nl
3,Tessebel,Tessaaatje,Jeminee ik word nu pas wakker wat is dit. Zo l...,nl
4,Thessa,juradoscrime,@mayravdzwaag hij was helemaal niet leuk en ik...,nl
5,De Westkrant,dewestkrant,"Het lijken wel kogelgaten, maar volgens de pol...",nl
6,Linda Pronk-Rijpstra,PronkRijpstra,@bvpuntcom ja hoor! Deze is van net ❤️ http://...,nl
7,PostNL,PostNL,"@mamarije30 Dat ga ik even voor je kijken, heb...",nl
8,hilda feenstra,hildafeenstra,"Na ruim 1,5 jaar vandaag mijn laatste werkdag ...",nl
9,Basten,GewoonBasten,@RowfeyVFX @Jerry_Kuijper heb ik ook gezegt! i...,nl


## Assignment 7
Count the number of tweets per user, and display the top 10 most-tweeting users.

In [66]:
name_df.groupBy('screen_name').count().orderBy('count', ascending = False).show(10)

+-------------+-----+
|  screen_name|count|
+-------------+-----+
|  GeaBijenhof|    8|
|    Hanglip68|    7|
|   news24hnld|    6|
|    nuswingen|    4|
|ItsEtienneZld|    4|
|          KLM|    4|
| nieuws_media|    4|
|      _Alphen|    3|
|oudersindekop|    3|
|    jongejan7|    3|
+-------------+-----+
only showing top 10 rows



## Word count in DataFrames

It is also possible to use DataFrames for less-structured data such as text. Here we show how you could do word count with DataFrames.

The following chained query contains a number of methods you haven't seen before, and we'll go through it line by line.

In [67]:
from pyspark.sql.functions import explode, split

spark \
    .read.text('../data/shakespeare.txt') \
    .select(explode(split("value", "\W+")).alias("word")) \
    .groupBy("word") \
    .count() \
    .orderBy("count", ascending=0).show()

+----+------+
|word| count|
+----+------+
|    |198753|
| the| 23288|
|   I| 22225|
| and| 18653|
|  to| 16373|
|  of| 15725|
|   a| 12796|
| you| 12186|
|  my| 10839|
|  in| 10016|
|   d|  8954|
|  is|  8414|
|that|  8343|
| not|  8038|
|  me|  7752|
|   s|  7487|
| And|  7457|
|with|  6802|
|  it|  6760|
|  be|  6412|
+----+------+
only showing top 20 rows



To see what happens here, we break it down into steps. First we read in the data file and inspect the DataFrame. It contains one column, called `value` by default.

In [68]:
swan_df = spark.read.text('../data/shakespeare.txt')
swan_df.show()

+--------------------+
|               value|
+--------------------+
|This is the 100th...|
|is presented in c...|
|Library of the Fu...|
|often releases Et...|
|                    |
|         Shakespeare|
|                    |
|*This Etext has c...|
|                    |
|<<THIS ELECTRONIC...|
|SHAKESPEARE IS CO...|
|PROVIDED BY PROJE...|
|WITH PERMISSION. ...|
|DISTRIBUTED SO LO...|
|PERSONAL USE ONLY...|
|COMMERCIALLY.  PR...|
|SERVICE THAT CHAR...|
|                    |
|*Project Gutenber...|
|in the presentati...|
+--------------------+
only showing top 20 rows



The column name `value` explains why it is mentioned inside the `split` function. Let's call the `select` method but omit `explode` and see what happens. Notice, that with `alias` we rename the column.

In [69]:
split_df = swan_df.select(split("value", "\W+").alias("word"))
split_df.show()

+--------------------+
|                word|
+--------------------+
|[This, is, the, 1...|
|[is, presented, i...|
|[Library, of, the...|
|[often, releases,...|
|                  []|
|       [Shakespeare]|
|                  []|
|[, This, Etext, h...|
|                  []|
|[, THIS, ELECTRON...|
|[SHAKESPEARE, IS,...|
|[PROVIDED, BY, PR...|
|[WITH, PERMISSION...|
|[DISTRIBUTED, SO,...|
|[PERSONAL, USE, O...|
|[COMMERCIALLY, PR...|
|[SERVICE, THAT, C...|
|                  []|
|[, Project, Guten...|
|[in, the, present...|
+--------------------+
only showing top 20 rows



Looking at the schema, we can see that `word` is actually an array of strings:

In [70]:
split_df.printSchema()

root
 |-- word: array (nullable = true)
 |    |-- element: string (containsNull = true)



Instead, we would like to have a row for each word, which is where [`explode`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode) comes in. It has a similar meaning as `flatMap` in Spark RDDs. It gets rid of lists:

In [71]:
swan_df.select(explode(split("value", "\W+")).alias("word")).show()

+-----------+
|       word|
+-----------+
|       This|
|         is|
|        the|
|      100th|
|      Etext|
|       file|
|  presented|
|         by|
|    Project|
|  Gutenberg|
|        and|
|         is|
|  presented|
|         in|
|cooperation|
|       with|
|      World|
|    Library|
|        Inc|
|       from|
+-----------+
only showing top 20 rows



### User-defined functions

In the previous example we used the built-in split function. It is also possible to define and use a custom user-defined function, or UDF. We'll show an example for the phone stock DataFrame first:

In [73]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

exp_udf = udf(lambda price: "Expensive" if price >= 500 else "Inexpensive", StringType())

phone_df.withColumn("cost", exp_udf(phone_df['unit_price'])).show()

+-------------+-------+-----+----------+-----------+
|        model|  brand|stock|unit_price|       cost|
+-------------+-------+-----+----------+-----------+
|     iPhone 6|  Apple|    6|     549.0|  Expensive|
|    iPhone 6s|  Apple|    5|     585.0|  Expensive|
|     iPhone 7|  Apple|   11|     739.0|  Expensive|
|        Pixel| Google|    8|     859.0|  Expensive|
|     Pixel XL| Google|    2|     959.0|  Expensive|
|    Galaxy S7|Samsung|   10|     539.0|  Expensive|
|    Galaxy S6|Samsung|    5|     414.0|Inexpensive|
|    Galaxy A5|Samsung|    7|     297.0|Inexpensive|
|Galaxy Note 7|Samsung|    0|     841.0|  Expensive|
+-------------+-------+-----+----------+-----------+



In this manner, we can apply specialized function, like tokenizers, on DataFrames. However, we first must register them as UDFs and cannot simply define them inline with lambda functions like we can with RDDs.

Below we define a very simple tokenizer, just as an example. It uses Python's string `split`, and also lowers the case of the text.

In [78]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

def my_tokenize(s):
    s = s.lower()
    words = s.split()
    return words

returnType = ArrayType(StringType())

tokenize_udf = udf(my_tokenize, returnType)

## Assignment 8
Use the `my_tokenize` function from the last cell to count words on the Shakespeare DataFrame `swan_df` instead of usng the `split` function. Display the top 10 most occurring words.

In [94]:
swan_df.select(explode(tokenize_udf("value")).alias("word")).groupBy("word").count().orderBy("count", ascending=False).show(10)

+----+-----+
|word|count|
+----+-----+
| the|27549|
| and|26037|
|   i|19540|
|  to|18700|
|  of|18010|
|   a|14383|
|  my|12455|
|  in|10671|
| you|10630|
|that|10487|
+----+-----+
only showing top 10 rows

